---

# Stop here

In [ ]:
!pip install pymilvus dspy openai nltk

In [17]:
import pandas as pd
import numpy as np

In [18]:
json_df = pd.read_csv('json_data_df.csv')
json_df.head()
json_df.info()
json_df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3154 entries, 0 to 3153
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      3154 non-null   int64  
 1   doc_id          3154 non-null   int64  
 2   index           3154 non-null   int64  
 3   link_path       2658 non-null   object 
 4   abstract        2656 non-null   object 
 5   all_authors     3154 non-null   object 
 6   pdf_path        2600 non-null   object 
 7   title           3154 non-null   object 
 8   publisher       3136 non-null   object 
 9   year_published  3154 non-null   int64  
 10  volume          2243 non-null   object 
 11  date_uploaded   2170 non-null   object 
 12  keywords        3154 non-null   object 
 13  start           986 non-null    float64
 14  end             985 non-null    float64
 15  paragraphs      3154 non-null   object 
 16  status          3154 non-null   int64  
 17  exception       3154 non-null   i

,Unnamed: 0,doc_id,index,year_published,start,end,status,exception
count,3154.000000,3154.000000,3154.000000,3154.000000,986.000000,985.000000,3154.000000,3154.000000
mean,1576.500000,2617.158529,2617.158529,2000.374445,175.803245,186.081218,1.891249,0.085606
std,910.625701,1523.436884,1523.436884,11.542785,133.080365,132.217983,0.311376,0.643809
min,0.000000,2.000000,2.000000,1922.000000,2.000000,3.000000,1.000000,0.000000
25%,788.250000,1311.250000,1311.250000,1992.000000,64.000000,75.000000,2.000000,0.000000
50%,1576.500000,2706.500000,2706.500000,1998.000000,148.500000,160.000000,2.000000,0.000000
75%,2364.750000,3747.500000,3747.500000,2009.000000,264.000000,272.000000,2.000000,0.000000
max,3153.000000,6185.000000,6185.000000,2023.000000,634.000000,646.000000,2.000000,5.000000


In [3]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility, db
import pandas as pd
import numpy as np
import ast

In [9]:

# Connect to Milvus server
connections.connect(host='milvus', port='19530')

# Load the JSON data into a DataFrame
json_data = pd.read_csv('json_data_df.csv')

# Load the embeddings data into a DataFrame
embedding_data = pd.read_csv('embedding_data_df.csv')

# Ensure there are no hidden NaN values in the embeddings
embedding_data = embedding_data.dropna(subset=['text-embedding-3-large', 'text-embedding-3-small'])

# Convert the embeddings from strings to lists of floats
embedding_data['text-embedding-3-large'] = embedding_data['text-embedding-3-large'].apply(lambda x: np.array(list(map(float, ast.literal_eval(x)))))
embedding_data['text-embedding-3-small'] = embedding_data['text-embedding-3-small'].apply(lambda x: np.array(list(map(float, ast.literal_eval(x)))))

# Merge the JSON data with the embeddings data on the 'doc_id' column
merged_data = pd.merge(json_data, embedding_data, on='doc_id')

# Define collection schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="doc_id", dtype=DataType.INT64),
    FieldSchema(name="index", dtype=DataType.INT64),
    FieldSchema(name="link_path", dtype=DataType.VARCHAR, max_length=2150),
    FieldSchema(name="abstract", dtype=DataType.VARCHAR, max_length=3209),
    FieldSchema(name="all_authors", dtype=DataType.VARCHAR, max_length=875),
    FieldSchema(name="pdf_path", dtype=DataType.VARCHAR, max_length=212),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=251),
    FieldSchema(name="publisher", dtype=DataType.VARCHAR, max_length=96),
    FieldSchema(name="year_published", dtype=DataType.INT64),
    FieldSchema(name="volume", dtype=DataType.VARCHAR, max_length=5),
    FieldSchema(name="date_uploaded", dtype=DataType.VARCHAR, max_length=10),
    FieldSchema(name="keywords", dtype=DataType.VARCHAR, max_length=285),
    FieldSchema(name="start", dtype=DataType.FLOAT),
    FieldSchema(name="end", dtype=DataType.FLOAT),
    # FieldSchema(name="paragraphs", dtype=DataType.VARCHAR, max_length=10000),
    FieldSchema(name="status", dtype=DataType.INT64),
    FieldSchema(name="exception", dtype=DataType.INT64),
    # FieldSchema(name="text_embedding_large", dtype=DataType.FLOAT_VECTOR, dim=3072),
    FieldSchema(name="text_embedding_small", dtype=DataType.FLOAT_VECTOR, dim=1536)
]

schema = CollectionSchema(fields, "Document data collection with embeddings")
collection_name = "doc_data"

# Drop existing collection if it exists
if utility.has_collection(collection_name):
    utility.drop_collection(collection_name)

# Create collection
collection = Collection(collection_name, schema)

# Filter out rows with missing or NaN values in any of the required columns
filtered_data = merged_data.dropna(subset=['doc_id', 'index', 'title', 'year_published', 'keywords', 'paragraphs', 'status', 'exception', 'text-embedding-3-large', 'text-embedding-3-small'])

# Insert data into the collection
collection.insert([
    filtered_data.index.tolist(),  # id
    filtered_data['doc_id'].tolist(),  # doc_id
    filtered_data['index'].tolist(),  # index
    filtered_data['link_path'].fillna('').tolist(),  # link_path
    filtered_data['abstract'].fillna('').tolist(),  # abstract
    filtered_data['all_authors'].tolist(),  # all_authors
    filtered_data['pdf_path'].fillna('').tolist(),  # pdf_path
    filtered_data['title'].tolist(),  # title
    filtered_data['publisher'].fillna('').tolist(),  # publisher
    filtered_data['year_published'].tolist(),  # year_published
    filtered_data['volume'].fillna('').tolist(),  # volume
    filtered_data['date_uploaded'].fillna('').tolist(),  # date_uploaded
    filtered_data['keywords'].tolist(),  # keywords
    filtered_data['start'].fillna(0).tolist(),  # start
    filtered_data['end'].fillna(0).tolist(),  # end
    # filtered_data['paragraphs'].tolist(),  # paragraphs
    filtered_data['status'].tolist(),  # status
    filtered_data['exception'].tolist(),  # exception
    # filtered_data['text-embedding-3-large'].tolist(),  # text_embedding_large
    filtered_data['text-embedding-3-small'].tolist()  # text_embedding_small
])

index_params = {
    "metric_type": "COSINE",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 56}
}
collection.create_index(field_name="text_embedding_small",index_params=index_params, index_name="text_embedding_small_index")
# Load the collection
collection.load()

print(f"Created collection with {collection.num_entities} entities")
connections.disconnect("default")

Created collection with 2967 entities


In [7]:
max_lengths = {}
for column in json_data.columns:
    if json_data[column].dtype == 'object':
        max_lengths[column] = json_data[column].dropna().apply(len).max()
    else:
        max_lengths[column] = None

for column, max_length in max_lengths.items():
    if max_length is not None:
        print(f"The maximum length of any element in the {column} column is: {max_length}")

The maximum length of any element in the link_path column is: 2141
The maximum length of any element in the abstract column is: 3209
The maximum length of any element in the all_authors column is: 875
The maximum length of any element in the pdf_path column is: 212
The maximum length of any element in the title column is: 251
The maximum length of any element in the publisher column is: 96
The maximum length of any element in the volume column is: 5
The maximum length of any element in the date_uploaded column is: 10
The maximum length of any element in the keywords column is: 285
The maximum length of any element in the paragraphs column is: 911088
